In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
import os
# 한글 지원
os.putenv('NLS_LANG', '.UTF8')

In [3]:
def ipchecker():    
    import urllib 
    import re
    
    pattern = re.compile(b'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}')
    dyndns = urllib.request.urlopen('http://checkip.dyndns.org').read()
    
    wan = re.search(pattern, dyndns).group().decode('utf-8')
    
    return wan

In [4]:
# base db_connect param

local_ip = '200.1.220.251:1521/xe' # 원내에서 접속할 db_ip
outside_ip = '61.35.152.103:1521/xe' # 원밖에서 접속할 db_ip
localhost = 'localhost:1521/xe'

db_id = 'begopa' 
db_pwd = 'begopa1'

academy_ip = '61.35.152.66'
db_ip = local_ip if academy_ip == ipchecker() else outside_ip

# db_ip = localhost # 집에서 본인의 개인 컴퓨터dptj  오라클에 접속할때는

In [5]:
import pickle
# load object ::
# :: file_nm
# :: columns and data_type
with open("./pickle/col2dict.pickle","rb") as fr:
    col2dict = pickle.load(fr)
with open("./pickle/table_nm_set.pickle","rb") as fr:
    table_nm_set = pickle.load(fr)

In [10]:
path = "./sg-data/"
desc_path = path + 'desc/'
col_path = path + 'col/'
origin_path = path + 'origin/'

file_list = os.listdir(origin_path)

In [11]:
# _, _, _, table_col_set_ = extract_table_columns_set(file_list[0]) # _, :: 임시로 쓰고 버리는 값.

In [12]:
# 자치구별 상권 변화지표 db 에서 한글 컬럼으로 가져오기
# oracle_to_df('자치구별 상권변화지표')
def load_to_df(kr_tbl_nm):
    for key, value in table_nm_set.items():
        if kr_tbl_nm == value:
            table_nm = key
            columns = [ arr[0] for arr in col2dict[key] ]
           
    import cx_Oracle
    import pandas as pd
    
    db = cx_Oracle.connect(db_id, db_pwd, db_ip) 
        
    sql = 'select * from ' + table_nm

    df = pd.read_sql(sql=sql, con = db) 
    df.columns = columns
    db.close
    
    return df

In [13]:
import re
import os

file_list = os.listdir(origin_path)
for file in file_list:
    kr_tbl_nm = re.findall('(\d\d).*\((.*)\)',file)
    print(kr_tbl_nm[0][0], kr_tbl_nm[0][1])

00 상권-영역
01 상권-추정유동인구
02 상권-집객시설
03 상권배후지-추정유동인구
04 상권배후지-집객시설
05 상권배후지-상주인구
06 상권-상주인구
07 상권배후지-아파트
08 상권배후지-점포
09 상권-상권변화지표
10 행정동별 상권변화지표
11 상권-추정매출
12 상권-직장인구
13 상권배후지-소득소비
14 상권배후지-직장인구
15 자치구별 상권변화지표
16 상권-아파트
18 상권-점포
19 상권배후지-추정매출


In [14]:
def multi_columns_merging(sg, sg_hl):
    a_df = load_to_df( sg )
    b_df = load_to_df( sg_hl )
    all_df = pd.merge(left=a_df, right=b_df, how='left', \
                  on=['기준_년_분기_코드','상권_코드'], sort=False)
#     del a_df, b_df
    all_df2 = all_df.drop(all_df.filter(regex='연령대|시간대'),axis=1)
    return all_df2.dropna(axis=0)

col = ['요일']
target=['연령대','시간대']
def drop_filter(df1,target):
    df_a=df1.drop(df1.filter(regex='|'.join(target)),axis=1)
    return df_a 

def multi_merge(a_df, b_df):
    all_df = pd.merge(left=a_df, right=b_df, how='left', on=['기준_년_분기_코드','상권_코드'], sort=False)
    all_df = all_df.drop(all_df.filter(regex='연령대|시간대'),axis=1)
    return all_df.dropna(axis=0)

def multi_columns_merging2(df, sg):
    a_df = load_to_df( sg )
    all_df = pd.merge(left=df, right=a_df, how='left', \
                  on=['기준_년_분기_코드','상권_코드'], sort=False)
#     del a_df, b_df
    all_df2 = all_df.drop(all_df.filter(regex='연령대|시간대'),axis=1)
    return all_df2.dropna(axis=0)

In [21]:
df0=load_to_df( '상권-영역' )
# df0.head()
# df0.info()

In [18]:
df15=load_to_df( '자치구별 상권변화지표' )
# df15.info()
df15.head()

,기준_년_분기_코드,시군구_코드,시군구_코드_명,상권_변화_지표,상권_변화_지표_명,운영_영업_개월_평균,폐업_영업_개월_평균,서울_운영_영업_개월_평균,서울_폐업_영업_개월_평균
0,2020_1,11740,강동구,LL,다이나믹,106,50,110,54
1,2020_1,11710,송파구,LH,상권확장,108,54,110,54
2,2020_1,11680,강남구,LL,다이나믹,102,50,110,54
3,2020_1,11650,서초구,HH,정체,118,54,110,54
4,2020_1,11620,관악구,LL,다이나믹,108,52,110,54


In [19]:
df15_1=df15[["시군구_코드",'시군구_코드_명']]
# df15_1.head()

In [20]:
map_df=df0.merge(df15_1, on='시군구_코드').drop_duplicates()
map_df.head()

,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,엑스좌표_값,와이좌표_값,시군구_코드,행정동_코드,시군구_코드_명
0,R,전통시장,1001453,낙성대시장,196121,442084,11620,11620585,관악구
28,R,전통시장,1001454,봉천제일종합시장,195147,442413,11620,11620595,관악구
56,R,전통시장,1001452,중부시장,196125,442779,11620,11620575,관악구
84,R,전통시장,1001460,관악신사시장,192646,442975,11620,11620685,관악구
112,R,전통시장,1001461,신림종합시장,192665,442804,11620,11620685,관악구


In [22]:
pop_df1=multi_columns_merging('상권-추정유동인구', '상권-상주인구')
# pop_df1   #유동, 상주 2962

In [23]:
target3=['월요','화요','수요','목요','금요']
target4=['토요','일요']
def add_week2(df,target3,target4):
    df['주중_유동인구_수']=df.filter(regex='|'.join(target3)).sum(axis=1)
    df['주말_유동인구_수']=df.filter(regex='|'.join(target4)).sum(axis=1)
    df_a=df.drop(df.filter(regex='|'.join(target3+target4)),axis=1)
    return df_a

# hl을 먼저 합해라!

In [25]:
pop_df2=add_week2(pop_df1,target3,target4)
pop_df2.head()

,기준_년_분기_코드,상권_코드,총_유동인구_수,남성_유동인구_수,여성_유동인구_수,총_상주인구_수,남성_상주인구_수,여성_상주인구_수,총_가구_수,아파트_가구_수,비_아파트_가구_수,주중_유동인구_수,주말_유동인구_수
0,2019_1,1000001,118266,61268,56997,1806.0,851.0,955.0,785.0,0.0,785.0,86815,31451
1,2019_1,1000002,246619,147358,99261,841.0,383.0,458.0,775.0,8.0,767.0,156385,90234
2,2019_1,1000003,144480,80073,64407,1214.0,682.0,532.0,805.0,0.0,805.0,107665,36815
3,2019_1,1000012,166390,70339,96051,2586.0,1163.0,1423.0,1087.0,111.0,976.0,121377,45018
4,2019_1,1000013,65518,33437,32080,249.0,147.0,102.0,166.0,0.0,166.0,46747,18769


In [26]:
# 비율 구하기
def ratio(df,col1,col2):
    total = df[col1]+df[col2]
    total2 = df[col1]/total*100
    return total2

In [27]:
pop_df2['남성_유동인구_비율']=ratio(pop_df2,'남성_유동인구_수', '여성_유동인구_수')
pop_df2['여성_유동인구_비율']=ratio(pop_df2,'여성_유동인구_수', '남성_유동인구_수')
pop_df2['주중_유동인구_비율']=ratio(pop_df2,'주중_유동인구_수', '주말_유동인구_수')
pop_df2['주말_유동인구_비율']=ratio(pop_df2,'주말_유동인구_수', '주중_유동인구_수')

In [30]:
pop_df2.columns

Index(['기준_년_분기_코드', '상권_코드', '총_유동인구_수', '남성_유동인구_수', '여성_유동인구_수', '총_상주인구_수',
       '남성_상주인구_수', '여성_상주인구_수', '총_가구_수', '아파트_가구_수', '비_아파트_가구_수',
       '주중_유동인구_수', '주말_유동인구_수', '남성_유동인구_비율', '여성_유동인구_비율', '주중_유동인구_비율',
       '주말_유동인구_비율', '상권_구분_코드', '상권_구분_코드_명', '상권_코드_명', '엑스좌표_값', '와이좌표_값',
       '시군구_코드', '행정동_코드', '시군구_코드_명'],
      dtype='object')

In [29]:
pop_df2=pop_df2.merge(map_df, on='상권_코드')

In [35]:
pop_df2.columns.to_list()
pop_df2=pop_df2[['기준_년_분기_코드', '상권_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드_명', '엑스좌표_값', '와이좌표_값',
                   '시군구_코드', '행정동_코드', '시군구_코드_명' ,'총_유동인구_수', '남성_유동인구_수', '여성_유동인구_수', '총_상주인구_수',
                   '남성_상주인구_수', '여성_상주인구_수', '총_가구_수', '아파트_가구_수', '비_아파트_가구_수',
                   '주중_유동인구_수', '주말_유동인구_수', '남성_유동인구_비율', '여성_유동인구_비율', '주중_유동인구_비율','주말_유동인구_비율']]
pop_df2.head()   # 영역(좌표) ,유동, 상주, 2962

,기준_년_분기_코드,상권_코드,상권_구분_코드,상권_구분_코드_명,상권_코드_명,엑스좌표_값,와이좌표_값,시군구_코드,행정동_코드,시군구_코드_명,...,여성_상주인구_수,총_가구_수,아파트_가구_수,비_아파트_가구_수,주중_유동인구_수,주말_유동인구_수,남성_유동인구_비율,여성_유동인구_비율,주중_유동인구_비율,주말_유동인구_비율
0,2019_1,1000001,A,골목상권,계동길,198799,453610,11110,11110600,종로구,...,955.0,785.0,0.0,785.0,86815,31451,51.805691,48.194309,73.406558,26.593442
1,2019_2,1000001,A,골목상권,계동길,198799,453610,11110,11110600,종로구,...,955.0,785.0,0.0,785.0,118416,48780,49.100185,50.899815,70.824661,29.175339
2,2019_1,1000002,A,골목상권,난계로27길,201996,452630,11110,11110710,종로구,...,458.0,775.0,8.0,767.0,156385,90234,59.751276,40.248724,63.411578,36.588422
3,2019_2,1000002,A,골목상권,난계로27길,201996,452630,11110,11110710,종로구,...,458.0,775.0,8.0,767.0,183408,119707,59.096847,40.903153,60.507728,39.492272
4,2019_1,1000003,A,골목상권,돈화문로11가길,198977,452902,11110,11110615,종로구,...,532.0,805.0,0.0,805.0,107665,36815,55.421512,44.578488,74.518965,25.481035


In [36]:
pop_df3=multi_columns_merging2(pop_df2,'상권-직장인구')
# pop_df3 # 영역(좌표) ,유동, 상주,직장인구 2960

In [25]:
# pop_df4[pop_df4['상권_코드']=='1000013']    #2개
# df16[df16['상권_코드']=='1000013']    #없음

In [49]:
df16 = load_to_df('상권-아파트')
# df16
len(set(pop_df3['상권_코드'].unique())- set(df16['상권_코드'].unique()))    #159
p_apt_df=pop_df3.merge(df16, on=['기준_년_분기_코드','상권_코드'], how='outer').drop_duplicates()
p_apt_df1=p_apt_df.drop(p_apt_df.filter(regex='아파트_면적_|아파트_가격_'),axis=1).drop_duplicates()
p_apt_df1=p_apt_df1.fillna('0')

In [62]:
del_f=p_apt_df1[p_apt_df1['상권_구분_코드']=='0'].index
p_apt_df2=p_apt_df1.drop(del_f)

In [64]:
# p_apt_df2   #유동, 상주,직장인구, 아파트 2960

In [69]:
df2=load_to_df('상권-집객시설')
# df2   #2924

In [86]:
p_apt_df3=p_apt_df2.merge(df2, on=['기준_년_분기_코드','상권_코드'], how='outer').drop_duplicates()
p_apt_df3=p_apt_df3.fillna('0')        #2986
len(p_apt_df3[p_apt_df3['상권_구분_코드']=='0'])
p_apt_df3[p_apt_df3['상권_구분_코드']=='0']
del_f2=p_apt_df3[p_apt_df3['상권_구분_코드']=='0'].index
p_apt_df3=p_apt_df3.drop(del_f2)

In [88]:
# p_apt_df3   # #유동, 상주,직장인구, 아파트, 집객시설 2960

In [89]:
df11=load_to_df('상권-추정매출')
df18=load_to_df('상권-점포')

In [90]:
df11=df11.drop(df11.filter(regex='시간대|연령대|요일'),axis=1).drop_duplicates()
df11.columns

Index(['기준_년_분기_코드', '상권_코드', '서비스_업종_코드', '서비스_업종_코드_명', '점포수', '당월_매출_금액',
       '당월_매출_건수', '주중_매출_비율', '주말_매출_비율', '남성_매출_비율', '여성_매출_비율', '주중_매출_금액',
       '주말_매출_금액', '남성_매출_금액', '여성_매출_금액', '주중_매출_건수', '주말_매출_건수', '남성_매출_건수',
       '여성_매출_건수'],
      dtype='object')

In [91]:
df18.columns

Index(['기준_년_분기_코드', '상권_코드', '서비스_업종_코드', '서비스_업종_코드_명', '점포_수', '유사_업종_점포_수',
       '개업_율', '개업_점포_수', '폐업_률', '폐업_점포_수', '프랜차이즈_점포_수'],
      dtype='object')

In [92]:
df18=df18.rename(columns={'점포_수':'점포수'})

In [94]:
service_df=df11.merge(df18, on=['기준_년_분기_코드', '상권_코드', '서비스_업종_코드', '서비스_업종_코드_명','점포수']).drop_duplicates()
service_df.head()
# service_df.info()

,기준_년_분기_코드,상권_코드,서비스_업종_코드,서비스_업종_코드_명,점포수,당월_매출_금액,당월_매출_건수,주중_매출_비율,주말_매출_비율,남성_매출_비율,...,주중_매출_건수,주말_매출_건수,남성_매출_건수,여성_매출_건수,유사_업종_점포_수,개업_율,개업_점포_수,폐업_률,폐업_점포_수,프랜차이즈_점포_수
0,2019_2,1001496,CS300043,전자상거래업,7,20931734,13998,71,29,45,...,9842,4156,8977,5016,7,0,0,14,1,0
1,2019_2,1001496,CS300035,인테리어,4,2000367097,48419,69,31,23,...,32895,15524,11553,36497,4,0,0,25,1,0
2,2019_2,1001496,CS300032,가전제품,8,2301647043,11720,68,33,43,...,8250,3470,4776,6524,8,0,0,0,0,0
3,2019_2,1001496,CS300028,화초,6,1356873015,33215,71,29,52,...,27570,5645,14364,16873,6,0,0,0,0,0
4,2019_2,1001496,CS300027,섬유제품,7,2130125149,62318,62,38,18,...,38868,23450,10911,51407,7,0,0,0,0,0


In [35]:
# p_apt_df3   # 유동, 상주,직장인구, 아파트, 집객시설 2960
# service_df      # 추정매출, 점포

In [36]:
# service_df.filter(regex='_비율')
# service_df.info()

In [95]:
def sg_svc_category_merge(df):
    grouped = df.groupby(['기준_년_분기_코드','상권_코드'])
    grouped_part1 = grouped.count() # 각 상권별 서비스 업종 코드수    
    grouped_part2 = grouped.sum() # 각 상권별 서비스 업종 분류수    
    join_grouped = grouped_part1.iloc[:,:1].join(grouped_part2, on=['기준_년_분기_코드', '상권_코드'], how='inner')
    return join_grouped

In [96]:
service_df_a = sg_svc_category_merge(service_df).reset_index(['상권_코드','기준_년_분기_코드'])
service_df_a=service_df_a.rename(columns={'서비스_업종_코드':'서비스_업종_개수'})
service_df_a.head()

,기준_년_분기_코드,상권_코드,서비스_업종_개수,점포수,당월_매출_금액,당월_매출_건수,주중_매출_비율,주말_매출_비율,남성_매출_비율,여성_매출_비율,...,주중_매출_건수,주말_매출_건수,남성_매출_건수,여성_매출_건수,유사_업종_점포_수,개업_율,개업_점포_수,폐업_률,폐업_점포_수,프랜차이즈_점포_수
0,2019_1,1000001,10,76,1804943415,53105,674,327,412,589,...,38687,14418,22652,25785,76,14,2,6,1,0
1,2019_1,1000002,14,62,5973375050,144623,1034,367,715,687,...,91396,53227,52384,90036,62,0,0,7,1,0
2,2019_1,1000003,18,80,2705497654,72125,1524,276,950,853,...,57178,14947,31624,31389,80,0,0,0,0,0
3,2019_1,1000004,14,34,1353059435,114386,1062,338,814,586,...,85784,28602,61922,52433,34,0,0,0,0,0
4,2019_1,1000005,11,40,2229198550,65160,620,481,514,586,...,35606,29554,27631,35000,40,0,0,47,2,0


In [97]:
all_df_a=p_apt_df3.merge(service_df_a, on=['기준_년_분기_코드','상권_코드']).drop_duplicates()

In [99]:
all_df_a.head()

,기준_년_분기_코드,상권_코드,상권_구분_코드,상권_구분_코드_명,상권_코드_명,엑스좌표_값,와이좌표_값,시군구_코드,행정동_코드,시군구_코드_명,...,주중_매출_건수,주말_매출_건수,남성_매출_건수,여성_매출_건수,유사_업종_점포_수,개업_율,개업_점포_수,폐업_률,폐업_점포_수,프랜차이즈_점포_수
0,2019_1,1000001,A,골목상권,계동길,198799,453610,11110,11110600,종로구,...,38687,14418,22652,25785,76,14,2,6,1,0
1,2019_2,1000001,A,골목상권,계동길,198799,453610,11110,11110600,종로구,...,42784,15848,25756,28683,74,32,4,31,2,0
2,2019_1,1000002,A,골목상권,난계로27길,201996,452630,11110,11110710,종로구,...,91396,53227,52384,90036,62,0,0,7,1,0
3,2019_2,1000002,A,골목상권,난계로27길,201996,452630,11110,11110710,종로구,...,89667,66066,66452,85631,64,19,2,0,0,0
4,2019_1,1000003,A,골목상권,돈화문로11가길,198977,452902,11110,11110615,종로구,...,57178,14947,31624,31389,80,0,0,0,0,0


In [106]:
df9=load_to_df('상권-상권변화지표')
df9=df9[['기준_년_분기_코드', '상권_코드', '운영_영업_개월_평균',
       '폐업_영업_개월_평균', '서울_운영_영업_개월_평균', '서울_폐업_영업_개월_평균','상권_변화_지표']]
# df9.columns

In [107]:
all_df_b=all_df_a.merge(df9, on=['기준_년_분기_코드', '상권_코드']).drop_duplicates()

In [108]:
all_df_b

,기준_년_분기_코드,상권_코드,상권_구분_코드,상권_구분_코드_명,상권_코드_명,엑스좌표_값,와이좌표_값,시군구_코드,행정동_코드,시군구_코드_명,...,개업_율,개업_점포_수,폐업_률,폐업_점포_수,프랜차이즈_점포_수,운영_영업_개월_평균,폐업_영업_개월_평균,서울_운영_영업_개월_평균,서울_폐업_영업_개월_평균,상권_변화_지표
0,2019_1,1000001,A,골목상권,계동길,198799,453610,11110,11110600,종로구,...,14,2,6,1,0,88,57,107,53,LH
1,2019_2,1000001,A,골목상권,계동길,198799,453610,11110,11110600,종로구,...,32,4,31,2,0,89,57,108,53,LH
2,2019_1,1000002,A,골목상권,난계로27길,201996,452630,11110,11110710,종로구,...,0,0,7,1,0,116,47,107,53,HL
3,2019_2,1000002,A,골목상권,난계로27길,201996,452630,11110,11110710,종로구,...,19,2,0,0,0,115,48,108,53,HL
4,2019_1,1000003,A,골목상권,돈화문로11가길,198977,452902,11110,11110615,종로구,...,0,0,0,0,0,113,56,107,53,HH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2944,2019_2,1001258,D,발달상권,등나무근린공원 주변,206032,460099,11350,11350625,노원구,...,25,1,31,2,0,88,51,108,53,LL
2945,2019_1,1001259,D,발달상권,서울 강북구 수유역_3,202401,460138,11305,11305630,강북구,...,53,4,122,4,0,111,51,107,53,HL
2946,2019_2,1001259,D,발달상권,서울 강북구 수유역_3,202401,460138,11305,11305630,강북구,...,79,7,100,1,2,111,52,108,53,HL
2947,2019_1,1001260,D,발달상권,창동역,204324,461465,11320,11320514,도봉구,...,53,5,56,9,0,105,51,107,53,LL


In [144]:
# all_df_b.to_csv('sg-data1.csv', index=False)

In [109]:
all_df_b1=all_df_b[['기준_년_분기_코드', '시군구_코드', '행정동_코드',
                               '상권_코드', '총_유동인구_수', '남성_유동인구_수',
                               '여성_유동인구_수', '총_상주인구_수', '남성_상주인구_수', '여성_상주인구_수', '총_가구_수', '아파트_가구_수',
                               '비_아파트_가구_수', '주중_유동인구_수', '주말_유동인구_수', '남성_유동인구_비율', '여성_유동인구_비율',
                               '주중_유동인구_비율', '주말_유동인구_비율', '총_직장_인구_수', '남성_직장_인구_수', '여성_직장_인구_수',
                               '아파트_단지_수', '아파트_평균_면적', '아파트_평균_시가', '집객시설_수', '관공서_수', '은행_수',
                               '종합병원_수', '일반_병원_수', '약국_수', '유치원_수', '초등학교_수', '중학교_수', '고등학교_수',
                               '대학교_수', '백화점_수', '슈퍼마켓_수', '극장_수', '숙박_시설_수', '공항_수', '철도_역_수',
                               '버스_터미널_수', '지하철_역_수', '버스_정거장_수', '점포수',
                               '당월_매출_금액', '당월_매출_건수', '주중_매출_비율', '주말_매출_비율', '남성_매출_비율', '여성_매출_비율',
                               '주중_매출_금액', '주말_매출_금액', '남성_매출_금액', '여성_매출_금액', '주중_매출_건수', '주말_매출_건수',
                               '남성_매출_건수', '여성_매출_건수', '유사_업종_점포_수', '개업_율', '개업_점포_수', '폐업_률',
                               '폐업_점포_수', '프랜차이즈_점포_수', '운영_영업_개월_평균', '폐업_영업_개월_평균', '서울_운영_영업_개월_평균',
                               '서울_폐업_영업_개월_평균', '상권_변화_지표']]

In [45]:
#머신러닝 - 예측 모델

In [110]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import pandas as pd

In [111]:
X=all_df_b1.iloc[:,:-1]
y=all_df_b1.iloc[:,-1:]

In [117]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
#                                                     test_size=0.3, 
                                                    stratify=y,
                                                    random_state=42)

In [118]:
X_train.shape   , X_test.shape , y_train.shape, y_test.shape

((2211, 70), (738, 70), (2211, 1), (738, 1))

In [119]:
# DecisionTreeClassifier 객체 생성
dt_clf = DecisionTreeClassifier(random_state=42)

# 학습수행
dt_clf.fit(X_train, y_train)

dt_clf.score(X_train, y_train), dt_clf.score(X_test, y_test)

(1.0, 0.9688346883468835)

In [120]:
# 학습이 완료된 DecisionTreeClassifier 객체에서 테스트 데이터 세트로 예측 수행
y_pred = dt_clf.predict(X_test)

In [121]:
from sklearn.metrics import accuracy_score
print('예측정확도 : {0: .4f}'.format(accuracy_score(y_test, y_pred)))

예측정확도 :  0.9688


In [122]:
from sklearn.model_selection import cross_val_score

cross_val_score(dt_clf, X_test, y_test, cv=10).mean()

0.9470381340244354

In [123]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [124]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 학습 진행
forest = RandomForestClassifier(n_estimators=1000, max_depth=10)
forest.fit(X_train, y_train)

# 예측
y_pred = forest.predict(X_test)
print(y_pred)
print(list(y_test))

#Random forest 정확도 측정
print(forest.score(X_train, y_train),forest.score(X_test, y_test))


# 정확도 확인
# accuracy_score(실제 레이블 세트, 예측 레이블 세트) : 정확도를 측정하기 위한 사이킷런의 함수
print('정확도 :', accuracy_score(y_test, y_pred))

c:\programdata\miniconda3\envs\hoseo\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


['LL' 'LH' 'LH' 'HL' 'HL' 'HH' 'LL' 'HH' 'HH' 'HL' 'LL' 'LL' 'LH' 'HH'
 'LL' 'LL' 'LL' 'LL' 'HL' 'LH' 'HH' 'HH' 'HH' 'HH' 'LL' 'LH' 'HH' 'HH'
 'HH' 'LH' 'HH' 'HH' 'LH' 'HH' 'LL' 'HL' 'HH' 'LL' 'HL' 'HH' 'HH' 'LL'
 'LL' 'LH' 'HH' 'LL' 'LL' 'LL' 'HH' 'LH' 'LL' 'HH' 'LH' 'LL' 'LL' 'LL'
 'LL' 'HH' 'HH' 'LL' 'HL' 'LL' 'HL' 'LL' 'HH' 'LH' 'LH' 'LL' 'LH' 'HH'
 'HH' 'LH' 'HL' 'HL' 'HH' 'LL' 'LH' 'HH' 'HH' 'HL' 'HH' 'LL' 'LH' 'HL'
 'HL' 'LL' 'LL' 'LL' 'LL' 'LL' 'LH' 'HL' 'LL' 'HH' 'LL' 'LH' 'LL' 'LH'
 'HL' 'LL' 'HH' 'LL' 'HL' 'LH' 'LL' 'LL' 'HH' 'HH' 'HH' 'LL' 'HH' 'LL'
 'LL' 'HH' 'LL' 'HH' 'LH' 'HH' 'HH' 'LL' 'HH' 'HH' 'HH' 'LL' 'HL' 'HH'
 'LL' 'HH' 'LL' 'HL' 'LL' 'HH' 'HL' 'LL' 'LL' 'LL' 'LL' 'HH' 'HH' 'HH'
 'HH' 'LH' 'HH' 'LH' 'LL' 'LL' 'LL' 'HH' 'HH' 'LH' 'LL' 'LL' 'HL' 'HH'
 'LH' 'LH' 'LL' 'HH' 'LH' 'HH' 'LL' 'LL' 'HH' 'HH' 'LL' 'HL' 'HL' 'HH'
 'HL' 'LL' 'LL' 'LL' 'HH' 'HH' 'HH' 'LH' 'LL' 'HH' 'HH' 'HH' 'HH' 'LL'
 'HL' 'HH' 'LL' 'LH' 'HL' 'HH' 'LL' 'LH' 'LH' 'HH' 'HH' 'HH' 'HH' 'HL'
 'HL' 

In [125]:
from sklearn.model_selection import cross_val_score
cross_val_score(forest, X_test, y_test, cv=10).mean()

c:\programdata\miniconda3\envs\hoseo\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\programdata\miniconda3\envs\hoseo\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\programdata\miniconda3\envs\hoseo\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\programdata\miniconda3\envs\hoseo\lib\site-packages\sklearn\model_selection\_validation.py:

0.9498333950388744

In [126]:
from sklearn.ensemble import GradientBoostingClassifier

In [138]:
gbc = GradientBoostingClassifier(n_estimators=4,max_depth=10)
gbc.fit(X_train, y_train)

c:\programdata\miniconda3\envs\hoseo\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


GradientBoostingClassifier(max_depth=10, n_estimators=4)

In [139]:
gbc.score(X_train, y_train), gbc.score(X_test, y_test)

(1.0, 0.9634146341463414)

In [142]:
from sklearn.model_selection import cross_val_score
cross_val_score(gbc, X_test, y_test, cv=10).mean()

c:\programdata\miniconda3\envs\hoseo\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
c:\programdata\miniconda3\envs\hoseo\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
c:\programdata\miniconda3\envs\hoseo\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
c:\programdata\miniconda3\envs\hoseo\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samp

0.9511106997408367

In [111]:
# from sklearn.svm import SVC 

In [115]:
# best_score = 0

# for gamma in [0.001, 0.01, 0.1, 1, 10, 100]: 
#     for C in [0.001, 0.01, 0.1, 1, 10, 100]: 
#         # 매개변수의 각 조합에 대해 SVC를 훈련시킵니다 
#         svm = SVC(gamma=gamma, C=C)
#         svm.fit(X_train, y_train) 
#         score = svm.score(X_test, y_test) # 테스트 세트로 평가
    
#         if score > best_score: # 점수가 더 높으면 매개변수와 함께 기록 
#             best_score = score 
#             best_parameters = {'C': C, 'gamma': gamma}

# print("최고 점수: {:.2f}".format(best_score))
# print("최적 파라미터: {}".format(best_parameters))